---
title: "Cuánto te afecta la semilla al resultado final"
description: "La semilla afecta a la aleatoriedad del ajuste del modelo. Esto puede generar una incertidumbre que no siempre se tiene en cuenta."
description-meta: "La semilla afecta a la aleatoriedad del ajuste del modelo. Esto puede generar una incertidumbre que no siempre se tiene en cuenta."
author: "Leonardo Hansa"
date: "2025-04-12"
categories: [exploraciones]
execute: 
  echo: true
  message: false
  warning: false
freeze: true
---


## Comentarios iniciales

En [Cuartil](https://open.spotify.com/episode/0Bl4iZq1o5s2Kd5WESm2Cu?si=LgiczmLkTfOpc0vKREYI9w) mencionamos que cambiar la semilla puede cambiar la métrica de ajuste de tu modelo.

Voy a ajustar un RandomForest en algún conjunto de datos varias veces. 

El objetivo es ver que, cada vez que lo ejecuto, la predicción cambia, por lo que hay cierta incertidumbre cada vez que ajustas un modelo de estos. 


In [ ]:
#| label: libs
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Preparación de datos

Los datos no me importan especialmente. Cargo unos de **sklearn** que me ha sugerido ChatGPT. Los separo en train y test. Esta separación la dejo fija: voy a estudiar cómo afecta la aleatoriedad del modelo, no la de los datos.


In [ ]:
#| label: data
data = fetch_california_housing(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Ajuste de los modelos


In [ ]:
#| label: loop-model
num_trials = 100
l_mse = []
df_preds = pd.DataFrame()

for i in range(num_trials):
  print(f'Iteración {i+1}/{num_trials}')
  random_state = np.random.randint(0, 10000)

  model = RandomForestRegressor(
    random_state=random_state, 
    n_jobs=-1
  )

  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  l_mse.append(mean_squared_error(y_test, y_pred))

  df_preds[f'pred_{i}'] = y_pred

## Visualización

### Métrica de ajuste


In [ ]:
plt.hist(l_mse, bins=20)

## Predicciones

In [ ]:
import seaborn as sns

df_long = df.reset_index().melt(id_vars='index', var_name='modelo', value_name='pred')
df_long = df_long.rename(columns={'index': 'id'})

subset_ids = df_long['id'].unique()[:100]
subset = df_long[df_long['id'].isin(subset_ids)]

plt.figure(figsize=(20, 6))
sns.boxplot(data=subset, x='id', y='pred', showfliers=False)
plt.xlabel('ID de observación (subconjunto)')
plt.ylabel('Predicción')
plt.title('Boxplot de predicciones por observación (100 primeras)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()